#Ini wajib dirun

In [1]:
# grader-required-cell

import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
root_dir = '/content/drive/Shareddrives/Capstone/MakeUpMate/DataGambar/cleaned_labeled_image'
class_size = 10
start_from = 1

#Ini untuk buat dataset

In [ ]:
def create_train_val_dirs(root_path, class_size, start_from):

  os.makedirs(os.path.join(root_path, "Train"))
  os.makedirs(os.path.join(root_path, "Validation"))


  #Create train dirs
  for i in range(start_from, class_size+1):
    os.makedirs(os.path.join(root_path, "Train/" + f"{i}"))

  #Create validation dirs
  for i in range(start_from, class_size+1):
    os.makedirs(os.path.join(root_path, "Validation/" + f"{i}"))


In [ ]:
try:
  create_train_val_dirs(root_path=root_dir, class_size=class_size, start_from=start_from)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [ ]:
train_list_dir = os.listdir(os.path.join(root_dir, "Train"))
validation_list_dir = os.listdir(os.path.join(root_dir, "Validation"))
source_list_dir = os.listdir(os.path.join(root_dir))
source_list_dir.remove("Train")
source_list_dir.remove("Validation")

print("Train Directories:" + f"{train_list_dir}\n")
print("Validation Directories:" + f"{validation_list_dir}\n")
print("Source Directories:" + f"{source_list_dir}\n")

Train Directories:['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

Validation Directories:['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

Source Directories:['7', '10', '3', '9', '2', '1', '4', '8', '6', '5']



In [ ]:
import math

def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

  list_source = os.listdir(SOURCE_DIR)
  list_source_rand = random.sample(list_source, len(list_source))
  size_train = int(len(list_source)*SPLIT_SIZE)
  #split train
  for i in range(size_train):
    if os.path.getsize(os.path.join(SOURCE_DIR, list_source_rand[i])) == 0:
      print(f"{list_source_rand[i]} is zero length, so ignoring.")
      continue
    copyfile(os.path.join(SOURCE_DIR, list_source_rand[i]), os.path.join(TRAINING_DIR, list_source_rand[i]))

  for i in range(size_train, len(list_source)):
    if os.path.getsize(os.path.join(SOURCE_DIR, list_source_rand[i])) == 0:
      print(f"{list_source_rand[i]} is zero length, so ignoring.")
      continue
    copyfile(os.path.join(SOURCE_DIR, list_source_rand[i]), os.path.join(VALIDATION_DIR, list_source_rand[i]))


In [ ]:
#Check if directory empty
for i in range(0, class_size):
  #Train
  if len(os.listdir(os.path.join(root_dir, "Train/"+f"{train_list_dir[i]}"))) > 0:
    for file in os.scandir(os.path.join(root_dir, "Train/"+f"{train_list_dir[i]}")):
      os.remove(file.path)
  #Validation
  if len(os.listdir(os.path.join(root_dir, "Validation/"+f"{validation_list_dir[i]}"))) > 0:
    for file in os.scandir(os.path.join(root_dir, "Validation/"+f"{validation_list_dir[i]}")):
      os.remove(file.path)

split_size = .8

# Run the function
for i in range(start_from, class_size+1):
  source_dir = os.path.join(root_dir, f"{i}")
  training_dir = os.path.join(root_dir, "Train/"+f"{i}")
  validation_dir = os.path.join(root_dir, "Validation/"+f"{i}")
  split_data(source_dir, training_dir, validation_dir, split_size)

# Check that the number of images matches the expected output

# Your function should perform copies rather than moving images so original directories should contain unchanged images
for i in range(start_from, class_size+1):
  source_dir = os.path.join(root_dir, f"{i}")
  print(f"\nOriginal {i} directory has {len(os.listdir(source_dir))} images")

# Training and validation splits
for i in range(start_from, class_size+1):
  training_dir = os.path.join(root_dir, "Train/"+f"{i}")
  validation_dir = os.path.join(root_dir, "Validation/"+f"{i}")
  print(f"\nTraining {i} directory has {len(os.listdir(training_dir))} images")
  print(f"Validation {i} directory has {len(os.listdir(validation_dir))} images")


Original 1 directory has 166 images

Original 2 directory has 348 images

Original 3 directory has 345 images

Original 4 directory has 239 images

Original 5 directory has 189 images

Original 6 directory has 165 images

Original 7 directory has 76 images

Original 8 directory has 143 images

Original 9 directory has 141 images

Original 10 directory has 111 images

Training 1 directory has 132 images
Validation 1 directory has 34 images

Training 2 directory has 278 images
Validation 2 directory has 70 images

Training 3 directory has 276 images
Validation 3 directory has 69 images

Training 4 directory has 191 images
Validation 4 directory has 48 images

Training 5 directory has 151 images
Validation 5 directory has 38 images

Training 6 directory has 132 images
Validation 6 directory has 33 images

Training 7 directory has 60 images
Validation 7 directory has 16 images

Training 8 directory has 114 images
Validation 8 directory has 29 images

Training 9 directory has 112 images
Va

#Ini untuk train dan modify model

In [4]:
training_dir = os.path.join(root_dir, "Train")
validation_dir = os.path.join(root_dir, "Validation")

In [5]:
# Set the learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 0.1 * 10**(epoch / 20))

In [6]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):

  train_datagen = ImageDataGenerator(rescale=1./255.)

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=20,
                                                      class_mode='categorical',
                                                      target_size=(150, 150))

  validation_datagen = ImageDataGenerator(rescale=1./255.)
  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=20,
                                                                class_mode='categorical',
                                                                target_size=(150, 150))
  ### END CODE HERE
  return train_generator, validation_generator

In [7]:
train_generator, validation_generator = train_val_generators(training_dir, validation_dir)

Found 1526 images belonging to 10 classes.
Found 389 images belonging to 10 classes.


In [10]:
def create_model():

  model = tf.keras.models.Sequential([ 
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  
  model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
                loss='categorical_crossentropy',
                metrics=['accuracy']) 
    
  return model

In [ ]:
# LR Tuning
model = create_model()

# Train the model
# Note that this may take some time.
history = model.fit(train_generator,
                    epochs=20,
                    verbose=1,
                    validation_data=validation_generator,
                    callbacks=[lr_schedule])

Epoch 1/10
77/77 [==============================] - 350s 5s/step - loss: 2.2481 - accuracy: 0.1776 - val_loss: 2.2043 - val_accuracy: 0.1851 - lr: 0.1000
Epoch 2/10
77/77 [==============================] - 343s 4s/step - loss: 2.0877 - accuracy: 0.2274 - val_loss: 1.9848 - val_accuracy: 0.2571 - lr: 0.1122
Epoch 3/10
77/77 [==============================] - 347s 5s/step - loss: 1.7326 - accuracy: 0.3768 - val_loss: 1.6373 - val_accuracy: 0.3111 - lr: 0.1259
Epoch 4/10
77/77 [==============================] - 319s 4s/step - loss: 1.4172 - accuracy: 0.4882 - val_loss: 1.2974 - val_accuracy: 0.5501 - lr: 0.1413
Epoch 5/10
77/77 [==============================] - 322s 4s/step - loss: 1.0937 - accuracy: 0.6206 - val_loss: 1.7870 - val_accuracy: 0.2468 - lr: 0.1585
Epoch 6/10
77/77 [==============================] - 347s 5s/step - loss: 0.9030 - accuracy: 0.6841 - val_loss: 1.0719 - val_accuracy: 0.6093 - lr: 0.1778
Epoch 7/10
77/77 [==============================] - 341s 4s/step - loss: 0.7

In [11]:
# Get the untrained model
model = create_model()

# Train the model
# Note that this may take some time.
history = model.fit(train_generator,
                    epochs=20,
                    verbose=1,
                    validation_data=validation_generator,)

Epoch 1/20
77/77 [==============================] - 222s 3s/step - loss: 2.0848 - accuracy: 0.2451 - val_loss: 1.8975 - val_accuracy: 0.3188
Epoch 2/20
77/77 [==============================] - 255s 3s/step - loss: 1.8423 - accuracy: 0.3250 - val_loss: 1.8561 - val_accuracy: 0.3008
Epoch 3/20
77/77 [==============================] - 253s 3s/step - loss: 1.6052 - accuracy: 0.4273 - val_loss: 1.5076 - val_accuracy: 0.4730
Epoch 4/20
77/77 [==============================] - 217s 3s/step - loss: 1.3973 - accuracy: 0.4954 - val_loss: 1.3390 - val_accuracy: 0.4910
Epoch 5/20
77/77 [==============================] - 215s 3s/step - loss: 1.1573 - accuracy: 0.5917 - val_loss: 1.0809 - val_accuracy: 0.6298
Epoch 6/20
77/77 [==============================] - 258s 3s/step - loss: 1.0059 - accuracy: 0.6481 - val_loss: 0.9738 - val_accuracy: 0.6375
Epoch 7/20
77/77 [==============================] - 215s 3s/step - loss: 0.8465 - accuracy: 0.7038 - val_loss: 0.8821 - val_accuracy: 0.7069
Epoch 8/20
77

In [14]:
import numpy as np

from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
import json
from json import JSONEncoder

import sys



uploaded=files.upload()

sample_arr = ""

for fn in uploaded.keys():
 
  path='/content/' + fn
  img=load_img(path, target_size=(150, 150))
  
  x=img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  # np.set_printoptions(threshold=sys.maxsize)

  # arr_str = np.array_str(images)

  # # print(arr_str)


  # with open('readme.txt', 'w') as f:
  #     for line in images:
  #         f.write(str(line))
  #         f.write('\n')
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])

class_result = 0
temp = 0
counter = 0

for i in classes[0]:
  counter+=1
  if i > temp:
    temp = i
    class_result = counter

print(class_result)
 

Saving 8BC7C220-E79E-41D6-B466-7BAFF3418D1C.jpg to 8BC7C220-E79E-41D6-B466-7BAFF3418D1C (1).jpg
1/1 [==============================] - 0s 22ms/step
[7.0601436e-03 1.8629929e-04 8.1943834e-01 9.6753821e-02 3.5978969e-02
 1.6689509e-02 1.6457019e-05 2.3847219e-02 1.7653472e-06 2.7544293e-05]
3


In [15]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model.h5')

In [16]:
!zip -r /content/saved_model.zip /content/saved_model
# change sample_data.zip to your desired download name Ex: nothing.zip
# change sample_data to your desired download folder name Ex: ner_data 

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/my_model.h5 (deflated 19%)


In [17]:
from google.colab import files
files.download("/content/saved_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Run dari sini kalau gamau train lagi

In [ ]:
load_model = tf.keras.models.load_model('saved_model/my_model.h5')

In [ ]:
load_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 32)      

In [ ]:
import numpy as np

from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
import json
from json import JSONEncoder

import sys

uploaded=files.upload()

sample_arr = ""

for fn in uploaded.keys():
 
  path='/content/' + fn
  img=load_img(path, target_size=(150, 150))
  
  x=img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  # np.set_printoptions(threshold=sys.maxsize)

  # arr_str = np.array_str(images)

  # # print(arr_str)


  # with open('readme.txt', 'w') as f:
  #     for line in images:
  #         f.write(str(line))
  #         f.write('\n')
  
  classes = load_model.predict(images, batch_size=10)
  
  print(classes[0])

class_result = 0
temp = 0
counter = 0

for i in classes[0]:
  counter+=1
  if i > temp:
    temp = i
    class_result = counter

print(class_result)
 